# งานที่ 5: เรียกโมเดล Bedrock สำหรับการสร้างรหัส

ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีใช้โมเดลภาษาขนาดใหญ่ (LLM) เพื่อสร้างโค้ดตามพรอมต์คำสั่งข้อความ

พรอมต์คำสั่งที่ใช้ในโน้ตบุ๊กนี้เป็นพรอมต์คำสั่งแบบ zero-shot เนื่องจากเราไม่ได้ให้ตัวอย่างข้อความอื่นนอกเหนือจากพรอมต์คำสั่งเอง

เพื่อแสดงให้เห็นถึงความสามารถในการสร้างรหัสของโมเดลใน Amazon Bedrock ให้คุณใช้กรณีการใช้งานของการสร้างรหัส คุณสำรวจการใช้ไคลเอ็นต์ Boto3 เพื่อสื่อสารกับ Amazon Bedrock API และให้ API ด้วยอินพุตที่ประกอบด้วยงาน คำสั่ง และอินพุตสำหรับโมเดลเบื้องหลังเพื่อสร้างเอาต์พุตโดยไม่ให้ตัวอย่างเพิ่มเติม จุดประสงค์ที่นี่คือเพื่อแสดงให้เห็นว่า LLM มีประสิทธิภาพในการเข้าใจงานที่อยู่ในมือได้อย่างง่ายดายและสร้างเอาต์พุตที่น่าสนใจได้อย่างไร


## สถานการณ์

คุณคือ Moe นักวิเคราะห์ข้อมูลที่ AnyCompany บริษัทต้องการทำความเข้าใจเกี่ยวกับประสิทธิภาพการขายสำหรับผลิตภัณฑ์ที่แตกต่างกันในช่วงปีที่ผ่านมา คุณได้รับชุดข้อมูลชื่อ sales.csv ชุดข้อมูลประกอบด้วยคอลัมน์ต่อไปนี้

- Date วันที่แบบ (ปปปป-ดด-วว)
- Product_ID (ตัวระบุที่ไม่ซ้ำกันสำหรับแต่ละผลิตภัณฑ์)
- Price (ราคาที่ขายในแต่ละผลิตภัณฑ์)


ในโน้ตบุ๊กนี้ คุณจะได้เรียนรู้วิธีสร้างรหัสสำหรับพรอมต์คำสั่งที่กำหนด คุณใช้ Meta LLama 3 โดยใช้ Amazon Bedrock API กับไคลเอ็นต์ Boto3

## งานที่ 5.1: การตั้งค่าสภาพแวดล้อม

ในงานนี้ ให้ตั้งค่าสภาพแวดล้อมของคุณ


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## งานที่ 5.2: การสร้างรหัส

ในงานนี้ ให้คุณเตรียมอินพุตสำหรับบริการ Amazon Bedrock เพื่อสร้างโปรแกรม python สำหรับกรณีการใช้งานของคุณ

#### การตั้งค่าสำหรับแล็บ - สร้างตัวอย่างข้อมูล sales.csv สำหรับแล็บนี้



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## งานที่ 5.3: การวิเคราะห์ยอดขายด้วยโปรแกรม Python ที่สร้างขึ้นของ Amazon Bedrock

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## งานที่ 5.4: เรียกโมเดล

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i> (ไม่จำเป็น) คัดลอกรหัสที่สร้างขึ้นจากเอาต์พุตที่พิมพ์ และรียกใช้รหัสที่สร้างขึ้นของ Bedrock ในเซลล์ด้านล่างเพื่อการตรวจสอบ


ตอนนี้คุณได้ทดลองใช้ SDK `boto3` ที่มีการเชื่อมต่อกับ Amazon Bedrock API การใช้ API นี้คุณจะสร้างโปรแกรม python เพื่อวิเคราะห์และแสดงภาพข้อมูลการขายที่กำหนด



### ลองด้วยตัวเอง

- เปลี่ยนพรอมต์คำสั่งตามฐานการใช้งานเฉพาะของคุณและประเมินเอาต์พุตของโมเดลต่างๆ
- เล่นด้วยความยาวโทเค็นเพื่อทำความเข้าใจเวลาแฝงและการตอบกลับของบริการ
- ใช้หลักการวิศวกรรมการโต้ตอบที่แตกต่างกันเพื่อให้ได้เอาต์พุตที่ดีขึ้น

### เก็บงาน

คุณทำโน้ตบุ๊กนี้เสร็จแล้ว หากต้องการย้ายไปยังส่วนถัดไปของแล็บ ให้ทำดังนี้

- ปิดไฟล์โน้ตบุ๊กนี้และไปต่อยัง **งานที่ 6**